# Mini-task-chem-1

## 0. Импорт библиотек и установка зависимостей

In [1]:
!pip install pubchempy rdkit-pypi tqdm pandas

In [2]:
!pip install numpy==1.26.4

In [3]:
import pubchempy as pcp
import pandas as pd
from tqdm.notebook import tqdm
from rdkit import Chem

## 1. Парсинг данных из PubChem по торговым названиям

In [17]:
drug_names = [
    "Aspirin", "Paracetamol", "Ibuprofen", "Amoxicillin", "Ciprofloxacin",
    "Metformin", "Omeprazole", "Simvastatin", "Losartan", "Lisinopril",
    "Atorvastatin", "Salbutamol", "Furosemide", "Levothyroxine", "Metoprolol",
    "Pantoprazole", "Dexamethasone", "Clopidogrel", "Prednisone", "Rosuvastatin"
]  # были выбраны в ручном режиме

results = []  # тут будут сохраняться результаты запросов

for name in tqdm(drug_names):
    try:
        cids = pcp.get_cids(name, 'name')
        if not cids:
            results.append({'compound_name': name, 'Error': 'No CID found'})
            continue

        for cid in cids:
            try:
                c = pcp.Compound.from_cid(cid)
                results.append({
                    'compound_name': name,
                    'CID': cid,
                    'canonical_smiles': c.canonical_smiles,
                    'isomeric_smiles': c.isomeric_smiles,
                    'inchi_key': c.inchikey,
                    'molecular_weight': c.molecular_weight,
                    'iupac_name': c.iupac_name
                })
            except Exception as inner:
                results.append({'compound_name': name, 'CID': cid, 'Error': str(inner)})

    except Exception as e:
        results.append({'compound_name': name, 'Error': str(e)})

df = pd.DataFrame(results)   # сохраняем результаты запросов в таблицу дата-фрейма
df.to_csv("pubchem_by_cid.csv", index=False)  # сохраняем дата-фрейм в csv файл
df.head()  # просматриваем первые несколько строк таблицы, чтобы посмотреть что мы сохранили

  0%|          | 0/20 [00:00<?, ?it/s]

,compound_name,CID,canonical_smiles,isomeric_smiles,inchi_key,molecular_weight,iupac_name,Error
0,Aspirin,2244.0,NaN,NaN,BSYNRYMUTXBXSQ-UHFFFAOYSA-N,180.16,2-acetyloxybenzoic acid,NaN
1,Paracetamol,1983.0,NaN,NaN,RZVAJINKPMORJF-UHFFFAOYSA-N,151.16,N-(4-hydroxyphenyl)acetamide,NaN
2,Ibuprofen,3672.0,NaN,NaN,HEFNNWSXXWATRW-UHFFFAOYSA-N,206.28,2-[4-(2-methylpropyl)phenyl]propanoic acid,NaN
3,Amoxicillin,33613.0,NaN,NaN,LSQZJLSUYDQPKJ-NJBDSQKTSA-N,365.4,"(2S,5R,6R)-6-[[(2R)-2-amino-2-(4-hydroxyphenyl...",NaN
4,Ciprofloxacin,2764.0,NaN,NaN,MYSWGUAQZAJSOK-UHFFFAOYSA-N,331.34,1-cyclopropyl-6-fluoro-4-oxo-7-piperazin-1-ylq...,NaN


Было замечено, что `canonical_smiles` часто `NaN`, при использовании библиотеки `rdkit` получить эти значения так же не удалось. Возможно, нужно было рассматривать другие препараты.

Несколько CID для одного названия выявлено не было.

## 2. Канонизация SMILES с помощью RDKit

In [11]:
def canonicalize_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        return Chem.MolToSmiles(mol, isomericSmiles=True, canonical=True)
    except:
        return None

df["canonicalized_smiles"] = df["canonical_smiles"].apply(canonicalize_smiles)
df.head()


,compound_name,CID,canonical_smiles,isomeric_smiles,inchi_key,molecular_weight,iupac_name,Error,canonicalized_smiles
0,Aspirin,2244.0,NaN,NaN,BSYNRYMUTXBXSQ-UHFFFAOYSA-N,180.16,2-acetyloxybenzoic acid,NaN,None
1,Paracetamol,1983.0,NaN,NaN,RZVAJINKPMORJF-UHFFFAOYSA-N,151.16,N-(4-hydroxyphenyl)acetamide,NaN,None
2,Ibuprofen,3672.0,NaN,NaN,HEFNNWSXXWATRW-UHFFFAOYSA-N,206.28,2-[4-(2-methylpropyl)phenyl]propanoic acid,NaN,None
3,Amoxicillin,33613.0,NaN,NaN,LSQZJLSUYDQPKJ-NJBDSQKTSA-N,365.4,"(2S,5R,6R)-6-[[(2R)-2-amino-2-(4-hydroxyphenyl...",NaN,None
4,Ciprofloxacin,2764.0,NaN,NaN,MYSWGUAQZAJSOK-UHFFFAOYSA-N,331.34,1-cyclopropyl-6-fluoro-4-oxo-7-piperazin-1-ylq...,NaN,None


Так как первоначально не было получено в таблицу `canonical_smiles`, то при использовании функции результат тоже оказался печальный == `NaN`.

In [12]:
df.shape # смотрим размерность - как было 20 препаратов, так и осталось, дубликатов нет

(20, 9)

## 3. Удаление дубликатов по InChIKey

В нашем случае дубликатов не было выявлено, но реализовать функцию или создание нового дата-сета без дубликатов - задача из мини-таски-1, поэтому вот:

In [13]:
df_no_duplicates = df.sort_values(by=["iupac_name", "molecular_weight"], na_position='last').drop_duplicates(subset="inchi_key", keep="first")
df_no_duplicates.head()


,compound_name,CID,canonical_smiles,isomeric_smiles,inchi_key,molecular_weight,iupac_name,Error,canonicalized_smiles
3,Amoxicillin,33613.0,NaN,NaN,LSQZJLSUYDQPKJ-NJBDSQKTSA-N,365.4,"(2S,5R,6R)-6-[[(2R)-2-amino-2-(4-hydroxyphenyl...",NaN,None
18,Prednisone,5865.0,NaN,NaN,XOFYZVNMUHMLCC-ZPOLXVRWSA-N,358.4,"(8S,9S,10R,13S,14S,17R)-17-hydroxy-17-(2-hydro...",NaN,None
19,Rosuvastatin,446157.0,NaN,NaN,BPRHUIZQVSMCRT-VEUZHWNKSA-N,481.5,"(E,3R,5S)-7-[4-(4-fluorophenyl)-2-[methyl(meth...",NaN,None
4,Ciprofloxacin,2764.0,NaN,NaN,MYSWGUAQZAJSOK-UHFFFAOYSA-N,331.34,1-cyclopropyl-6-fluoro-4-oxo-7-piperazin-1-ylq...,NaN,None
2,Ibuprofen,3672.0,NaN,NaN,HEFNNWSXXWATRW-UHFFFAOYSA-N,206.28,2-[4-(2-methylpropyl)phenyl]propanoic acid,NaN,None


## 4. Анализ пропущенных значений

In [15]:
missing = df_no_duplicates.isna().sum()  # счет пропущенных значений
percent = (missing / len(df_no_duplicates) * 100).round(2)  # подсчет процента, округление до 2 знаков
pd.DataFrame({'missing': missing, 'percent': percent}) # вывод подсчетов в виде таблицы


,missing,percent
compound_name,0,0.0
CID,0,0.0
canonical_smiles,10,100.0
isomeric_smiles,10,100.0
inchi_key,1,10.0
molecular_weight,1,10.0
iupac_name,1,10.0
Error,9,90.0
canonicalized_smiles,10,100.0


## 5. Добавление источника и сохранение итогового датасета

Само добавление данных не было реализовано, поэтому сразу запишем, что все данные использованные были взяты только из `PubChem`.

Сохраним результат в файл `final_dataset_mini_task_1.csv`.

In [16]:
df_no_duplicates["source"] = "PubChem"

final_df = df_no_duplicates[[
    "compound_name", "canonicalized_smiles", "inchi_key", "molecular_weight", "source"
]]

final_df.to_csv("final_dataset_mini_task_1.csv", index=False)
final_df.head()


,compound_name,canonicalized_smiles,inchi_key,molecular_weight,source
3,Amoxicillin,None,LSQZJLSUYDQPKJ-NJBDSQKTSA-N,365.4,PubChem
18,Prednisone,None,XOFYZVNMUHMLCC-ZPOLXVRWSA-N,358.4,PubChem
19,Rosuvastatin,None,BPRHUIZQVSMCRT-VEUZHWNKSA-N,481.5,PubChem
4,Ciprofloxacin,None,MYSWGUAQZAJSOK-UHFFFAOYSA-N,331.34,PubChem
2,Ibuprofen,None,HEFNNWSXXWATRW-UHFFFAOYSA-N,206.28,PubChem
